# 基于LLM微调的数学推理任务

In [1]:
import os 
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load
from datasets import load_dataset

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

/autodl-tmp/conda/Base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 设置代理
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
dataset = load_dataset("openai/gsm8k", "main")

In [4]:
def get_dataloader(prompt, batch_size=1):
    def preprocess(dataset, shuffle=True):
        def collate_fn(batch):
            questions =  [[{"role": "user", "content": prompt.format(text=item['question'])}] for item in batch]
            answers = [item["answer"] for item in batch]
            return questions, answers
        return DataLoader(
            dataset,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=collate_fn,
        )
    testset = dataset['test']
    return preprocess(testset, shuffle=False)

In [5]:
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

def extract_answer(answer):
    match = ANS_RE.search(answer)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS
    
def is_correct(reply, truth):
    answer = extract_answer(truth)
    assert answer != INVALID_ANS
    return reply == answer

In [6]:
def eval_process(model_name, tokenizer_name, prompt, batch_size=1, ratio=1, result_num=1):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
        cache_dir = '/autodl-tmp/cache'
    )
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    testloader = get_dataloader(prompt, batch_size)
    answers = []
    replies = [[] for _ in range(result_num)]
    # 以下过程参照huggingface上的qwen2.5模型示例
    for idx, (texts, truths) in tqdm(enumerate(testloader), total=int(len(testloader) * ratio)):
        if idx >= int(len(testloader) * ratio):
            break
        for i in range(result_num):

            texts = [tokenizer.apply_chat_template(
                text,
                tokenize=False,
                add_generation_prompt=True
            ) for text in texts]
            model_inputs = tokenizer(texts, return_tensors="pt", padding=True, padding_side='left').to(model.device)

            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=256
            )

            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]

            responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            replies[i].extend(responses)
        answers.extend(truths)
    
    total = 0
    correct = 0
    for i in range(len(answers)):
        total += 1
        candidates = []
        for j in range(result_num):
            candidates.append(extract_answer(replies[j][i]))
        from collections import Counter
        count = Counter(candidates)
        reply = count.most_common(1)[0][0]
        answer = answers[i]
        correct += is_correct(reply, answer)
    print(f"正确率: {(correct / total) * 100: .2f}%")

In [8]:
model_name = "output/qwen2_5-0_5b-instruct/v1-20241125-232429/checkpoint-450"
tokenizer_name = "Qwen/Qwen2.5-0.5B-Instruct"

prompt = '''You are a highly skilled mathematician capable of solving complex grade-school math problems step by step. Please read the problem carefully and solve it with clear reasoning. Follow these instructions:

- Identify the known information and the question being asked.
- Break the solution into logical steps, providing clear explanations for each.
- Show all calculations and intermediate results.
- Conclude with a final answer.

Output your response in the following format:
[Explanation and calculations]
#### [Final numerical answer]   

Here is the problem:
{text}'''

In [10]:
eval_process(model_name, tokenizer_name, prompt, batch_size=8, ratio=0.1, result_num=1)

100%|██████████| 16/16 [01:51<00:00,  6.97s/it]

正确率:  29.69%


In [11]:
eval_process(model_name, tokenizer_name, prompt, batch_size=8, ratio=0.1, result_num=3)

100%|██████████| 16/16 [05:16<00:00, 19.80s/it]

正确率:  0.00%


In [12]:
eval_process(model_name, tokenizer_name, prompt, batch_size=8, ratio=0.1, result_num=5)

100%|██████████| 16/16 [09:43<00:00, 36.46s/it]

正确率:  0.00%


In [1]:
# 导入swift框架进行微调
from swift.llm import (
    DatasetName, InferArguments, ModelType, SftArguments,
    infer_main, sft_main, app_ui_main
)

/autodl-tmp/conda/Base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO:swift] Successfully registered `/autodl-tmp/conda/Base/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] No vLLM installed, if you are using vLLM, you will get `ImportError: cannot import name 'get_vllm_engine' from 'swift.llm'`
[INFO:swift] No LMDeploy installed, if you are using LMDeploy, you will get `ImportError: cannot import name 'prepare_lmdeploy_engine_template' from 'swift.llm'`


In [2]:
model_type = ModelType.qwen2_5_0_5b_instruct
sft_args = SftArguments(
    model_type=model_type,
    dataset=['gsm8k-train.jsonl'],
    output_dir='output',
    max_length=4096)
result = sft_main(sft_args)
last_model_checkpoint = result['last_model_checkpoint']

[INFO:swift] Setting template_type: qwen2_5
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] Setting args.dataloader_num_workers: 1
[INFO:swift] output_dir: /root/autodl-fs/code/nlp_pj/pj6/output/qwen2_5-0_5b-instruct/v1-20241125-232429
[INFO:swift] Start time of running main: 2024-11-25 23:24:29.426308
[INFO:swift] args: SftArguments(model_type='qwen2_5-0_5b-instruct', model_id_or_path='qwen/Qwen2.5-0.5B-Instruct', model_revision='master', full_determinism=False, sft_type='lora', freeze_parameters=[], freeze_vit=False, freeze_parameters_ratio=0.0, additional_trainable_parameters=[], tuner_backend='peft', template_type='qwen2_5', output_dir='/root/autodl-fs/code/nlp_pj/pj6/output/qwen2_5-0_5b-instruct/v1-20241125-232429', add_output_dir_suffix=True, ddp_backend=None, ddp_find_unused_parameters=None, ddp_broadcast_buffers=None, ddp_timeout=1800, seed=42, resume_from_checkpoint=None, resume_only_model=False, ignore_data_skip=False, dtype='bf16', packing=False, train_backend='t

device_count: 1
rank: -1, local_rank: -1, world_size: 1, local_world_size: 1


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:swift] Loading the model using model_dir: /root/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
[INFO:swift] model.max_model_len: 32768
[INFO:swift] model.hf_device_map: {'': device(type='cuda', index=0)}
[INFO:swift] model_config: Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/root/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sli

{'loss': 0.61197513, 'acc': 0.85235035, 'grad_norm': 3.15869617, 'learning_rate': 4.17e-06, 'memory(GiB)': 3.21, 'train_speed(iter/s)': 0.202167, 'epoch': 0.0, 'global_step/max_steps': '1/462', 'percentage': '0.22%', 'elapsed_time': '4s', 'remaining_time': '35m 29s'}


Train:   1%|          | 5/462 [00:15<22:23,  2.94s/it]

...

Val: 100%|██████████| 74/74 [00:03<00:00, 21.11it/s]/it]


{'eval_loss': 0.43656504, 'eval_acc': 0.86965001, 'eval_runtime': 3.4886, 'eval_samples_per_second': 21.212, 'eval_steps_per_second': 21.212, 'epoch': 1.0, 'global_step/max_steps': '462/462', 'percentage': '100.00%', 'elapsed_time': '22m 29s', 'remaining_time': '0s'}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[INFO:swift] Saving model checkpoint to /root/autodl-fs/code/nlp_pj/pj6/output/qwen2_5-0_5b-instruct/v1-20241125-232429/checkpoint-462
Train: 100%|██████████| 462/462 [22:30<00:00,  2.92s/it]
[INFO:swift] last_model_checkpoint: /root/autodl-fs/code/nlp_pj/pj6/output/qwen2_5-0_5b-instruct/v1-20241125-232429/checkpoint-462
[INFO:swift] best_model_checkpoint: /root/autodl-fs/code/nlp_pj/pj6/output/qwen2_5-0_5b-instruct/v1-20241125-232429/checkpoint-462
[INFO:swift] images_dir: /root/autodl-fs/code/nlp_pj/pj6/output/qwen2_5-0_5b-instruct/v1-20241125-232429/images


{'train_runtime': 1349.9878, 'train_samples_per_second': 5.481, 'train_steps_per_second': 0.342, 'train_loss': 0.44557389, 'epoch': 1.0, 'global_step/max_steps': '462/462', 'percentage': '100.00%', 'elapsed_time': '22m 30s', 'remaining_time': '0s'}


[INFO:swift] End time of running main: 2024-11-25 23:47:16.806765
